In [ ]:
import pandas as pd

data = r"3993249.csv"

df = pd.read_csv(data)
df = df[df['STATION'] == 'USW00014935']
df.set_index('DATE', inplace=True)

df = df.loc['1964-04-01':]

In [ ]:
null_cols = df.isnull().sum()

#print(null_cols)

empty = null_cols[null_cols > 15000]

bad_columns = [c for c in list(empty.index) if 'WT' not in c]

#print(bad_columns)

condensed = df.drop(bad_columns, axis=1)

#print(condensed)

In [ ]:
condensed['TAVG'] = (condensed['TMAX'] + condensed['TMIN']) / 2

In [ ]:
weather_type_columns = [c for c in condensed.columns if 'WT' in c]
condensed[weather_type_columns] = condensed[weather_type_columns].fillna(0)
condensed

In [ ]:
condensed.to_csv('test.csv')

In [ ]:
import numpy as np

new_df = condensed.loc['1984-01-01':'2024-12-31']

columns_to_drop = ['ACMH', 'ACSH', 'FMTM', 'PGTM', 'WDF5', 'WSF5', 'SNOW', 'SNWD']
columns_to_keep = [c for c in new_df.columns if c not in columns_to_drop]

new_df = new_df[columns_to_keep]
new_df[['WDF1', 'WDF2', 'WSF1', 'WSF2']] = new_df[['WDF1', 'WDF2', 'WSF1', 'WSF2']].fillna(0)

# combine the two remaining wind measurements into one 
new_df['WDF12'] = np.pi * (new_df['WDF1'] + new_df['WDF2']) / 180.0
new_df['WSF12'] = new_df['WSF1'] + new_df['WSF2']

# also include the U and V components of the wind
new_df['UWSF12'] = new_df['WSF12'] * np.cos(new_df['WDF12'])
new_df['VWSF12'] = new_df['WSF12'] * np.sin(new_df['WDF12'])

# use hundredths of an inch for precipitation measurement
new_df['PRCP'] = new_df['PRCP'] * 100.0

new_df.insert(0, 'DAY', pd.to_datetime(new_df.index).day_of_year)
new_df['DAY'] = new_df['DAY'].apply(lambda x: x if x < 366 else 1)

new_df['DAY365'] = pd.to_datetime(new_df.index).day_of_year / 365.25
new_df['SIN'] = np.sin(2 * np.pi * new_df['DAY365'])
new_df['COS'] = np.cos(2 * np.pi * new_df['DAY365'])

new_df['TSPREAD'] = new_df['TMAX'] - new_df['TMIN']
new_df['WSPREAD'] = new_df['WSF12'] - new_df['AWND']

new_df = new_df.drop(['WDF1', 'WDF2', 'WSF1', 'WSF2'], axis=1)
new_df = new_df.ffill()

data_columns = ['DAY', 'AWND', 'PRCP', 'TMAX', 'TMIN', 'UWSF12', 'VWSF12']
derived_columns = [x for x in new_df.columns if x not in data_columns]

new_df = new_df[data_columns + derived_columns]

new_df = new_df.drop(['STATION', 'NAME'], axis=1)
new_df.to_csv('grand_island.csv')
new_df.drop(weather_type_columns, axis=1).to_csv('grand_island_nwt.csv')

In [ ]:
up_to_2000 = new_df.loc[:'2000-01-01']
up_to_2000

In [ ]:

ddd = up_to_2000.drop(weather_type_columns, axis=1).groupby(['DAY']).agg(['mean', 'std'])#.to_csv('grand_island_normals.csv')
ddd.columns = ['_'.join(col).strip() for col in ddd.columns.values]

#ddd.to_csv('grand_island_normals.csv')
total_means = new_df.drop(weather_type_columns, axis=1).agg(['mean', 'std'])

c = {}
for column in total_means.columns:
    if column == 'DAY':
        continue

    mean = total_means[column].loc['mean']
    std = total_means[column].loc['std']
    c[f'{column}_mean'] = mean
    c[f'{column}_std'] = std
    
print(c)

total_means = pd.DataFrame([c])
total_means.index = [9999]

pd.concat((ddd, total_means)).to_csv('grand_island_normals.csv')